In [ ]:
#@title  **Top 10 Central Conceptual Domains in the Quranic Network**



# --- [الخطوة 0: التثبيت والاستيراد] ---
!git clone https://github.com/NoorBayan/Amthal.git
%cd Amthal
from utility import *
from IPython.display import display, HTML, clear_output


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import io
from itertools import combinations



# --- [الخطوة 1: تحميل وتجهيز البيانات] ---
print("--- [Step 1: Loading Data] ---")
# ⚠️⚠️⚠️  الرجاء التأكد من أن هذا المسار صحيح في بيئتك
INSTANCES_FILE_PATH = '/content/Amthal/data/processed/instances.csv'
try:
    df = pd.read_csv(INSTANCES_FILE_PATH, sep='\t', encoding='utf-16')
    df['Dominant_Concept']=[Dominant_Concept_dic[i]['en'] for i in df.Dominant_Concept]
    df['Valence']=[Valence_dic[i]['ar'] for i in df.Valence]

    print(f"Data loaded successfully. Found {len(df)} instances.\n")
except FileNotFoundError:
    print(f"⚠️ ERROR: File not found at '{INSTANCES_FILE_PATH}'. Please check the path and try again.")
    exit()



# --- 2. بناء جدول الروابط (edge_weights) من بياناتك الحقيقية ---
# هذا يضمن أن بيانات الشبكة وبيانات الترميز تأتي من نفس المصدر
print("Building edge weights from instance data...")
grouped = df.groupby(['Sura_No', 'Aya_No'])['Dominant_Concept'].apply(list)
edges = []
for concepts in grouped:
    if len(concepts) > 1:
        for combo in combinations(set(concepts), 2):
            edges.append(tuple(sorted(combo)))

edge_weights = pd.DataFrame(edges, columns=['source', 'target'])
edge_weights = edge_weights.value_counts().reset_index(name='weight')
print("Edge weights built successfully.")


# --- 3. حساب كل المقاييس المطلوبة من بياناتك الحقيقية ---

# --- الجزء الأول: مقاييس الشبكة ---
print("Calculating network metrics...")
G = nx.from_pandas_edgelist(edge_weights, 'source', 'target', ['weight'])
degrees = dict(G.degree(weight='weight'))
betweenness = nx.betweenness_centrality(G, weight='weight', normalized=True)
network_metrics = pd.DataFrame.from_dict(degrees, orient='index', columns=['Weighted_Degree'])
network_metrics['Betweenness'] = network_metrics.index.map(betweenness)
network_metrics.index.name = 'Dominant_Concept'

# --- الجزء الثاني: مقاييس الترميز المجمعة ---
print("Aggregating annotation metrics...")
agg_metrics = df.groupby('Dominant_Concept').agg(
    Instance_Count=('Dominant_Concept', 'count'),
    Mean_Dominance_Score=('Dominance_Score', 'mean'),
    Std_Dominance_Score=('Dominance_Score', 'std'),
    Percent_Concrete=('Abstraction_Level', lambda x: (x == 'محسوس').sum() / len(x) * 100 if len(x) > 0 else 0)
)
agg_metrics['Std_Dominance_Score'] = agg_metrics['Std_Dominance_Score'].fillna(0)
agg_metrics['Dominance_Score (Mean ± SD)'] = agg_metrics.apply(
    lambda row: f"{row['Mean_Dominance_Score']:.2f} ± {row['Std_Dominance_Score']:.2f}", axis=1
)

# --- الجزء الثالث: توزيع الشحنة (Valence Distribution) ---
print("Calculating valence distributions...")
valence_dist = pd.crosstab(df['Dominant_Concept'], df['Valence'])
valence_dist['Valence_Distribution (Pos/Neg/Neu)'] = valence_dist.apply(
    lambda row: f"{row.get('إيجابي', 0)} / {row.get('سلبي', 0)} / {row.get('محايد', 0)}",
    axis=1
)

# --- 4. دمج كل البيانات في جدول واحد ---
print("Merging all metrics into the final table...")
final_table = network_metrics.merge(agg_metrics, left_index=True, right_index=True, how='left')
final_table = final_table.merge(valence_dist[['Valence_Distribution (Pos/Neg/Neu)']], left_index=True, right_index=True, how='left')
# .fillna() للتعامل مع المفاهيم التي قد تظهر في الشبكة ولكن ليس لها مثيلات (حالة نادرة)
final_table.fillna(0, inplace=True)


# --- 5. إضافة الأعمدة الوصفية والترتيب النهائي ---
structural_roles = {
    'طريق': 'Ethical & navigational hub', 'قوة': 'Theological & agential hub',
    'إدراك': 'Epistemological & cognitive hub', 'خراب': 'Consequential & negative hub',
    'مأوى': 'Hub for salvation & divine reward', 'بناء': 'Hub for order & social construction',
    'نور': 'Key sensory metaphor for guidance', 'ماء': 'Hub for life & purification',
    'حياة': 'Core ontological hub', 'تجارة': 'Key metaphor for divine transaction'
}
final_table['Structural_Role'] = final_table.index.map(structural_roles).fillna('Supportive Role')
final_table_sorted = final_table.sort_values(by='Weighted_Degree', ascending=False)
final_table_sorted.insert(0, 'Rank', range(1, len(final_table_sorted) + 1))


# --- 6. اختيار الأعمدة النهائية وتنسيقها للعرض ---
final_table_sorted.reset_index(inplace=True)
final_table_sorted.rename(columns={'index': 'Dominant_Concept'}, inplace=True)

concept_map_en = {
    'طريق': 'PATH', 'قوة': 'POWER', 'إدراك': 'PERCEPTION', 'خراب': 'RUIN',
    'مأوى': 'SHELTER', 'بناء': 'BUILDING', 'نور': 'LIGHT', 'ماء': 'WATER',
    'حياة': 'LIFE', 'تجارة': 'TRADE', 'خوف': 'FEAR', 'موت': 'DEATH',
    'ظلام': 'DARKNESS', 'نار': 'FIRE', 'خصائص مادية': 'MATERIALITY',
    'معاملات': 'TRANSACTION', 'شعور': 'FEELING', 'سلوكيات': 'BEHAVIOR',
    'علاقات اجتماعية': 'SOCIAL REL.', 'ابتلاء': 'TRIAL', 'تطهير وإصلاح': 'PURIFICATION'
}
final_table_sorted['Dominant_Concept'] = final_table_sorted['Dominant_Concept'].apply(
    lambda x: f"{concept_map_en.get(x, x.capitalize())} ({x})"
)

display_columns = [
    'Rank', 'Dominant_Concept', 'Instance_Count', 'Dominance_Score (Mean ± SD)',
    'Percent_Concrete', 'Valence_Distribution (Pos/Neg/Neu)',
    'Weighted_Degree', 'Betweenness', 'Structural_Role'
]

final_display = final_table_sorted[display_columns].head(10)

# تنسيق الأرقام النهائية
final_display['Percent_Concrete'] = final_display['Percent_Concrete'].map('{:.1f}%'.format)
final_display['Betweenness'] = final_display['Betweenness'].map('{:.3f}'.format)
# تحويل instance count إلى عدد صحيح
final_display['Instance_Count'] = final_display['Instance_Count'].astype(int)


# --- 7. عرض الجدول النهائي ---
print("\n" + "="*80)
print("--- Table 1: Top 10 Central Conceptual Domains in the Quranic Network ---")
print("="*80)
print(final_display.to_string(index=False))
print("="*80)

Cloning into 'Amthal'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 62 (delta 11), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (62/62), 721.01 KiB | 3.52 MiB/s, done.
Resolving deltas: 100% (11/11), done.
/content/Amthal
--- [Step 1: Loading Data] ---
Data loaded successfully. Found 4078 instances.

Building edge weights from instance data...
Edge weights built successfully.
Calculating network metrics...
Aggregating annotation metrics...
Calculating valence distributions...
Merging all metrics into the final table...

--- Table 1: Top 10 Central Conceptual Domains in the Quranic Network ---
 Rank            Dominant_Concept  Instance_Count Dominance_Score (Mean ± SD) Percent_Concrete Valence_Distribution (Pos/Neg/Neu)  Weighted_Degree Betweenness Structural_Role
    1                 Path (Path)             735                 4.25 ± 0.79             0.0%    